# 05 — Main Dashboard (Non-Regression)

**Goal:** Build clean Delta tables for dashboard visualizations (excluding regression outputs).

**Inputs (from previous notebooks):**
- `workspace.default.daily_efficiency`
- `workspace.default.hourly_efficiency`
- `workspace.default.priority_buildings`

**Outputs (this notebook):**
- `workspace.default.dashboard_daily_kpis`
- `workspace.default.dashboard_utility_summary`
- `workspace.default.dashboard_building_summary`
- `workspace.default.dashboard_hourly_patterns`
- `workspace.default.dashboard_priority_vs_nonpriority`


## **Load Tables**

In [0]:
# ------------------------------------------------------------
# Import Spark SQL functions
# ------------------------------------------------------------
from pyspark.sql import functions as F

# ------------------------------------------------------------
# Load DAILY efficiency table
# Contains building/day/utility level energy metrics
# ------------------------------------------------------------
daily_eff = spark.table("workspace.default.daily_efficiency")

# ------------------------------------------------------------
# Load HOURLY efficiency table
# Used for load-shape analysis
# ------------------------------------------------------------
hourly_eff = spark.table("workspace.default.hourly_efficiency")

# ------------------------------------------------------------
# Load PRIORITY buildings list
# Used to tag buildings for comparison dashboards
# ------------------------------------------------------------
priority = (
    spark.table("workspace.default.priority_buildings")
    .select("sitename")
    .distinct()
)

display(daily_eff.limit(100000))


## **Add is_priority Flag**

In [0]:
# ------------------------------------------------------------
# Add priority flag to DAILY dataset
#
# If building exists in priority table:
#   is_priority = 1
# Else:
#   is_priority = 0
# ------------------------------------------------------------
daily_eff2 = (
    daily_eff
    .join(priority.withColumn("is_priority", F.lit(1)),
          on="sitename",
          how="left")
    .fillna({"is_priority": 0})
)

# ------------------------------------------------------------
# Add same flag to HOURLY dataset
# ------------------------------------------------------------
hourly_eff2 = (
    hourly_eff
    .join(priority.withColumn("is_priority", F.lit(1)),
          on="sitename",
          how="left")
    .fillna({"is_priority": 0})
)

display(daily_eff2.select("sitename","day","utility","is_priority").limit(10))


## **Daily KPI Table**

In [0]:
# ------------------------------------------------------------
# DAILY KPI TABLE
#
# Purpose:
# Overall trendline dashboard.
#
# Each row = one day
#
# Metrics:
# - number of buildings reporting
# - total energy usage
# - average energy intensity
# ------------------------------------------------------------
dashboard_daily_kpis = (
    daily_eff2
    .groupBy("day")
    .agg(
        F.count("*").alias("n_rows"),                     # number of records
        F.countDistinct("sitename").alias("n_buildings"),# buildings active
        F.sum("total_usage").alias("total_usage"),       # total campus usage
        F.avg("energy_per_sqm").alias("avg_energy_per_sqm"),
        F.avg("square_meters").alias("avg_square_meters")
    )
    .orderBy("day")
)

# ------------------------------------------------------------
# Save as Delta table for dashboard
# ------------------------------------------------------------
dashboard_daily_kpis.write.mode("overwrite").format("delta") \
    .saveAsTable("workspace.default.dashboard_daily_kpis")

display(dashboard_daily_kpis.limit(20))


## **Utility Summary**

In [0]:
# ------------------------------------------------------------
# UTILITY SUMMARY TABLE
#
# Purpose:
# Compare Electricity vs Steam vs Cooling vs Gas
# Used for bar charts in dashboard.
# ------------------------------------------------------------
dashboard_utility_summary = (
    daily_eff2
    .groupBy("utility")
    .agg(
        F.countDistinct("sitename").alias("n_buildings"),
        F.count("*").alias("n_rows"),
        F.sum("total_usage").alias("total_usage"),
        F.avg("energy_per_sqm").alias("avg_energy_per_sqm")
    )
    .orderBy(F.col("total_usage").desc())
)

dashboard_utility_summary.write.mode("overwrite").format("delta") \
    .saveAsTable("workspace.default.dashboard_utility_summary")

display(dashboard_utility_summary)


## **Building Summary**

In [0]:
# ------------------------------------------------------------
# BUILDING SUMMARY TABLE
#
# Purpose:
# Rank buildings by energy intensity.
#
# Useful dashboard visuals:
# - Top worst buildings
# - Efficiency benchmarking
# ------------------------------------------------------------
dashboard_building_summary = (
    daily_eff2
    .groupBy("sitename")
    .agg(
        F.count("*").alias("n_rows"),
        F.first("campusname", ignorenulls=True).alias("campusname"),
        F.avg("square_meters").alias("avg_square_meters"),
        F.sum("total_usage").alias("total_usage"),
        F.avg("energy_per_sqm").alias("avg_energy_per_sqm"),
        F.expr("percentile_approx(energy_per_sqm, 0.95)")
            .alias("p95_energy_per_sqm")
    )
    .orderBy(F.col("avg_energy_per_sqm").desc())
)

dashboard_building_summary.write.mode("overwrite").format("delta") \
    .saveAsTable("workspace.default.dashboard_building_summary")

display(dashboard_building_summary.limit(25))


## **Priority vs Non-Priority Comparison**


In [0]:
# ------------------------------------------------------------
# PRIORITY vs NON-PRIORITY BUILDINGS
#
# Purpose:
# Compare performance of flagged buildings.
#
# Used for:
# - grouped bar charts
# - performance benchmarking
# ------------------------------------------------------------
dashboard_priority_vs_nonpriority = (
    daily_eff2
    .groupBy("is_priority","utility")
    .agg(
        F.countDistinct("sitename").alias("n_buildings"),
        F.sum("total_usage").alias("total_usage"),
        F.avg("energy_per_sqm").alias("avg_energy_per_sqm"),
        F.expr("percentile_approx(energy_per_sqm,0.90)")
            .alias("p90_energy_per_sqm")
    )
    .orderBy("utility","is_priority")
)

dashboard_priority_vs_nonpriority.write.mode("overwrite").format("delta") \
    .saveAsTable("workspace.default.dashboard_priority_vs_nonpriority")

display(dashboard_priority_vs_nonpriority)
